**Multi-objective optimization with BoTorch**

Expected Hyper-volume improvement

In [1]:
import pandas as pd
import numpy as np
import torch
import botorch
import gpytorch
import matplotlib.pyplot as plt

c:\Users\goura\anaconda3\envs\botorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Reading input data

In [6]:
filename = r'../data/olhs_run1.xlsx'
x_pd = pd.read_excel(filename, sheet_name='Initial Design (OLHS)', header=[0,1], index_col=[0])
y_pd = pd.read_excel(filename, sheet_name='bo_data', header=[0,1], index_col=[0])

# no normalization

dtype=torch.double

validation_idx = [1,7,15]

train_x_pd = x_pd.drop(validation_idx)
train_y_pd = y_pd.drop(validation_idx)

# which properties to read from labels
objective_properties = ['Polymer Solubility', 'Gelation Enthalpy', 'Shear Modulus']

# make torch tensors 
train_x = torch.tensor(train_x_pd.values, dtype=dtype)
train_y_list = []
for prop in objective_properties:
    train_y_list.append(
        torch.tensor(train_y_pd[prop].values, dtype=dtype)
    )

Define the models for regression

In [15]:
from botorch.models.gp_regression import SingleTaskGP
from botorch.models.model_list_gp_regression import ModelListGP
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch.utils.transforms import unnormalize, normalize

xmins = x_pd.min(axis=0).values
xmaxs = x_pd.max(axis=0).values
bounds = torch.tensor([np.array(xmins), np.array(xmaxs)], dtype=dtype)
train_x = normalize(train_x, bounds=bounds)

models = []
for data in train_y_list:
    models.append(
        SingleTaskGP(train_x, data, outcome_transform=Standardize(m=1))
    )
model = ModelListGP(*models)
mll = SumMarginalLogLikelihood(model.likelihood, model)

In [16]:
from botorch.optim.optimize import optimize_acqf, optimize_acqf_list
from botorch.acquisition.objective import GenericMCObjective
from botorch.utils.multi_objective.scalarization import get_chebyshev_scalarization
from botorch.utils.multi_objective.box_decompositions.non_dominated import (
    FastNondominatedPartitioning,
)
from botorch.acquisition.multi_objective.monte_carlo import (
    qExpectedHypervolumeImprovement,
    qNoisyExpectedHypervolumeImprovement,
)
from botorch.utils.sampling import sample_simplex

BATCH_SIZE = 4  # following tutorial
NUM_RESTARTS = 10   # tutorial
RAW_SAMPLES = 512   # tutorial

In [17]:
# function to optimize acquisition function
standard_bounds = torch.zeros(2, 8)
standard_bounds[1] = 1

ref_point = torch.tensor([18, 0.1, 0.01], dtype=dtype)

def optimize_qehvi_and_get_observation(model, train_x, train_obj, sampler):
    """Optimizes the qEHVI acquisition function, and returns a new candidate and observation."""
    # partition non-dominated space into disjoint rectangles
    with torch.no_grad():
        pred = model.posterior(normalize(train_x, bounds)).mean
    partitioning = FastNondominatedPartitioning(
        ref_point= ref_point,
        Y=pred,
    )
    acq_func = qExpectedHypervolumeImprovement(
        model=model,
        ref_point=ref_point,
        partitioning=partitioning,
        sampler=sampler,
    )
    # optimize
    candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=standard_bounds,
        q=BATCH_SIZE,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,  # used for intialization heuristic
        options={"batch_limit": 5, "maxiter": 200},
        sequential=True,
    )
    # observe new values
    new_x = unnormalize(candidates.detach(), bounds=bounds)
    # new_obj_true = problem(new_x)
    # new_obj = new_obj_true 

    return new_x #, new_obj, new_obj_true

In [ ]:
from botorch import fit_gpytorch_mll
from botorch.exceptions import BadInitialCandidatesWarning
from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.utils.multi_objective.box_decompositions.dominated import (
    DominatedPartitioning,
)
from botorch.utils.multi_objective.pareto import is_non_dominated

hvs_qehvi = []

# fit model
fit_gpytorch_mll(mll)

# define acquisition modules
sampler = SobolQMCNormalSampler(sample_shape=torch.Size([128]))

new_x_qehvi = optimize_qehvi_and_get_observation(model, train_x, train_y_list, sampler)

#need to define the current best to calculate the hypervolume with respect to